In [17]:
demand_path = "input"
output_path = "output"

In [28]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import itertools
import geopandas as gpd
import os

In [19]:
assert os.path.exists("%s/confluence_areas.gpkg" % demand_path)
assert os.path.exists("%s/confluence.osm.pbf" % output_path)

assert os.path.exists("%s/slots.csv" % output_path)
assert os.path.exists("%s/homes.gpkg" % demand_path)

# Relevant information

In [20]:
df_spatial = gpd.read_file("%s/homes.gpkg" % demand_path)

In [21]:
df_slots = pd.read_csv("%s/slots.csv" % output_path, sep = ";")

# Filter Confluence households

In [23]:
confluence_households = gpd.sjoin(
    df_spatial,
    gpd.read_file("%s/confluence_areas.gpkg" % demand_path),
    op = "within"
)["household_id"].unique()

In [24]:
df_confluence = df_slots[df_slots["household_id"].isin(confluence_households)]

# Network

In [25]:
import networkx as nx
import pyrosm 

In [26]:
osm = pyrosm.OSM("%s/confluence.osm.pbf" % output_path)
df_nodes, df_edges = osm.get_network(nodes = True, network_type = "driving")

df_nodes = df_nodes.to_crs("EPSG:2154")
df_edges = df_edges.to_crs("EPSG:2154")

graph = osm.to_graph(df_nodes, df_edges, graph_type="networkx")
df_nodes = df_nodes[df_nodes["id"].isin(graph.nodes)]

In [31]:
# Calculate distance matrix

nodes = list(graph.nodes)
df_distances = []

with tqdm(total = np.cumsum(np.arange(len(nodes)))[-1]) as progress:
    progress.update()
    
    for i in range(len(nodes)): 
        df_distances.append(dict(
            from_node = nodes[i], to_node = nodes[i],
            distance = 0.0
        ))
        
        for j in range(i + 1, len(nodes)):
            df_distances.append(dict(
                from_node = nodes[i], to_node = nodes[j],
                distance = nx.algorithms.shortest_path_length(graph, nodes[i], nodes[j], weight = "length")
            ))
            
            progress.update()

df_distances = pd.DataFrame.from_records(df_distances)

In [32]:
# Find household nodes
import sklearn
import sklearn.neighbors

node_index = sklearn.neighbors.KDTree(
    np.vstack([df_nodes["geometry"].x.values, df_nodes["geometry"].y.values]).T
)

df_households = df_slots.drop_duplicates("household_id")[["household_id", "x", "y"]]

df_households["location_id"] = df_nodes.iloc[node_index.query(
    np.vstack([df_households["x"].values, df_households["y"].values]).T
)[1].flatten()]["id"].values

In [33]:
pd.merge(df_confluence, df_households[["household_id", "location_id"]]).to_csv(
    "%s/vrp_deliveries.csv" % output_path, sep = ";"
)

df_distances.to_csv(
    "%s/vrp_distances.csv" % output_path, sep = ";"
)

In [34]:
# Write outnetwork

df_edges[
    df_edges["u"].isin(df_nodes["id"]) & df_edges["v"].isin(df_nodes["id"])
].to_file("%s/vrp_network.gpkg" % output_path, driver = "GPKG")

df_nodes.to_file("%s/vrp_nodes.gpkg" % output_path, driver = "GPKG")

In [35]:
df_nodes_file = df_nodes.copy()
df_nodes_file["x"] = df_nodes.geometry.x
df_nodes_file["y"] = df_nodes.geometry.y
df_nodes_file["location_id"] = df_nodes_file["id"]
df_nodes_file[["location_id", "x", "y"]].to_csv("%s/vrp_nodes.csv" % output_path, sep = ";")